<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0f3b7193646eb5c8fb27d34c7ce2827d30cc7c79d386670566c12a7e6fc7df52
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-21 10:24:40
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.35 C
-------------------
Today PnL: -80.11 K (-0.59%)
Current PnL: -33.95 L (-21.65%)
CY Booked + Current PnL: -19.66 L (-12.54%)
-------------------
Total profit:  1.15 L
Total loss:  -35.10 L
-------------------
Total Booked + Current PnL: 7.32 L (5.64%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.58%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 99.30 L (73.49%)
Deployed:  1.30 C
Current:  1.35 C
CAGR/XIRR %: 2.13%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.33,13.85,6.63,21.40,20035.0,36762.0,302188.0,5.69,51.0,X-LC,7.04,14.0,1.83,2.24,21.05,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,-0.23,-23.49,30.83,0.09,23770.0,-23677.0,77100.0,78.87,36.0,M-SC,4.97,253.0,-1.00,0.57,1.44,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.07,2.05,16.83,19.23,25842.0,3090.0,153550.0,19.88,52.0,L-LC,10.52,182.0,0.12,1.14,23.77,XY25,ATH,MINING
49,MASFIN,398.61,-2.19,-9.06,34.21,22.05,30481.0,-8880.0,89100.0,-21.41,36.0,H-SC,4.64,168.0,-0.29,0.66,29.51,XR,ATH,FINANCE
32,HCLTECH,1908.19,-0.55,8.90,13.72,23.83,36145.0,21522.0,263446.0,13.61,57.0,X-LC,6.33,13.0,0.60,1.95,26.40,X40,ATH,IT


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,4546.37,3.81,-31.61,183.14,93.63,126449.0,-31918.0,69045.0,-26.73,50.0,H-SC,16.48,143.0,-0.25,0.51,4.38,SR,ATH,CHEMICALS
38,INDIAMART,4810.62,2.90,-5.29,118.27,106.72,138154.0,-6524.0,116812.0,-53.11,50.0,H-SC,8.73,138.0,-0.05,0.86,16.85,AR,ATH,MISC
34,HONAUT,58357.33,1.51,-16.72,72.09,43.32,97783.0,-27232.0,135640.0,-28.43,51.0,X-SC,8.13,90.0,-0.28,1.00,4.15,X40N,ATH,ELECTRICAL
75,TMPV,600.00,1.18,-29.93,75.52,22.98,119531.0,-67621.0,158277.0,-26.74,36.0,X-LC,3.59,2.0,-0.57,1.17,1.18,XY24,NTT,AUTO
37,IEX,219.00,0.92,-10.76,66.55,48.63,118746.0,-21522.0,178432.0,-40.74,40.0,H-SC,13.48,137.0,-0.18,1.32,0.92,XR,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.00,-4.21,-41.01,69.49,-0.01,52394.0,-52407.0,75398.0,15334.78,29.0,M-SC,14.06,235.0,-1.00,0.56,0.00,OX40N,BTT,AC
31,HAVELLS,2069.16,-3.45,-16.78,59.19,32.48,161564.0,-55033.0,272958.0,-21.00,27.0,X-MC,5.30,69.0,-0.34,2.02,0.00,X40,ATH,ELECTRICAL
56,RAJOOENG,143.10,-3.09,-34.15,112.00,39.61,75600.0,-35000.0,67500.0,-55.45,49.0,H-SC,28.73,136.0,-0.46,0.50,9.26,AR,ATH,MISC
82,VOLTAS,1856.00,-2.83,0.67,44.23,45.20,85373.0,1278.0,193020.0,-9.09,29.0,X-MC,5.27,80.0,0.01,1.43,7.51,XY25,NTT,AC
78,UNITDSPR,1644.00,-2.71,-5.77,28.16,20.77,62133.0,-13502.0,220642.0,-13.95,28.0,X-MC,2.56,70.0,-0.22,1.63,0.75,X40N,NTT,BREWERIES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.25,-9.16,128.87,107.91,172848.0,-13518.0,134126.0,-26.23,34.0,M-SC,10.01,220.0,-0.08,0.99,0.01,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.16,-20.79,26.28,0.02,36335.0,-36299.0,138260.0,110.94,37.0,M-SC,8.78,240.0,-1.00,1.02,20.26,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.32,-22.61,70.60,32.03,72033.0,-29805.0,102030.0,-16.20,47.0,H-SC,6.43,157.0,-0.41,0.76,14.28,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-0.23,-23.49,30.83,0.09,23770.0,-23677.0,77100.0,78.87,36.0,M-SC,4.97,253.0,-1.00,0.57,1.44,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.30,-25.27,51.85,13.47,104483.0,-68157.0,201510.0,-69.28,38.0,H-SC,4.39,159.0,-0.65,1.49,2.46,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-0.07,2.05,16.83,19.23,25842.0,3090.0,153550.0,19.88,52.0,L-LC,10.52,182.0,0.12,1.14,23.77,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.60,-1.01,74.02,72.27,124160.0,-1710.0,167738.0,-15.15,34.0,M-LC,4.56,101.0,-0.01,1.24,5.10,XR,NTT,IT
38,INDIAMART,4810.62,2.90,-5.29,118.27,106.72,138154.0,-6524.0,116812.0,-53.11,50.0,H-SC,8.73,138.0,-0.05,0.86,16.85,AR,ATH,MISC
25,FINCABLES,1641.55,-1.25,-9.16,128.87,107.91,172848.0,-13518.0,134126.0,-26.23,34.0,M-SC,10.01,220.0,-0.08,0.99,0.01,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.80,-7.17,46.10,35.63,89691.0,-15017.0,194557.0,-20.28,38.0,H-MC,4.70,119.0,-0.17,1.44,8.31,AR,ATH,PHARMA
37,IEX,219.00,0.92,-10.76,66.55,48.63,118746.0,-21522.0,178432.0,-40.74,40.0,H-SC,13.48,137.0,-0.18,1.32,0.92,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.54,-32.29,132.17,57.19,291841.0,-105320.0,220807.0,-67.97,15.0,X-MC,12.93,58.0,-0.36,1.63,0.00,XY24,NTT,FMCG
43,ITC,452.00,0.21,-19.85,38.23,10.78,73632.0,-47715.0,192603.0,-52.19,17.0,X-LC,12.77,1.0,-0.65,1.43,0.21,X40,NTT,FMCG
58,RELAXO,1176.00,0.12,-52.69,221.93,52.31,152413.0,-76484.0,68676.0,-49.67,19.0,X-SC,9.13,92.0,-0.50,0.51,0.12,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.70,-43.10,137.63,35.20,100759.0,-55460.0,73210.0,-3.97,19.0,X-SC,18.18,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
53,PGHH,17907.65,0.00,-10.23,49.48,34.19,94843.0,-21840.0,191680.0,-36.94,22.0,X-MC,5.43,60.0,-0.23,1.42,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,0.21,-14.52,38.69,18.55,125095.0,-54911.0,323326.0,-26.21,44.0,X-LC,1.00,3.0,-0.44,2.39,10.00,X40,ATH,IT
35,ICICIGI,2252.93,-0.74,-2.11,21.92,19.34,42936.0,-4232.0,195877.0,-22.62,33.0,X-MC,2.18,71.0,-0.10,1.45,13.49,X40,ATH,INSURANCE
33,HINDUNILVR,2922.00,0.28,-7.19,22.47,13.67,53609.0,-18483.0,238580.0,-14.11,54.0,X-LC,2.26,9.0,-0.34,1.77,12.63,XY25,NTT,FMCG
20,DABUR,735.00,0.32,-0.93,45.06,43.70,110735.0,-2318.0,245750.0,-9.06,46.0,X-MC,2.27,73.0,-0.02,1.82,13.98,XY24,BTT,FMCG
45,JIOFIN,387.00,0.09,-14.52,45.63,24.48,104285.0,-38820.0,228545.0,-15.96,26.0,X-LC,2.46,37.0,-0.37,1.69,32.47,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.54,-32.29,132.17,57.19,291841.0,-105320.0,220807.0,-67.97,15.0,X-MC,12.93,58.0,-0.36,1.63,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-0.63,-23.58,103.45,55.47,192328.0,-57372.0,185914.0,-32.33,22.0,X-MC,15.99,63.0,-0.30,1.38,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.70,-43.10,137.63,35.20,100759.0,-55460.0,73210.0,-3.97,19.0,X-SC,18.18,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.41,-27.72,81.60,31.26,136109.0,-63968.0,166800.0,-66.25,24.0,X-MC,24.04,56.0,-0.47,1.23,0.0,X40N,ATH,IT
31,HAVELLS,2069.16,-3.45,-16.78,59.19,32.48,161564.0,-55033.0,272958.0,-21.00,27.0,X-MC,5.30,69.0,-0.34,2.02,0.0,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.45,-33.34,114.27,42.84,49520.0,-21670.0,43336.0,-56.47,33.0,X-SC,23.02,83.0,-0.44,0.32,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,0.12,-52.69,221.93,52.31,152413.0,-76484.0,68676.0,-49.67,19.0,X-SC,9.13,92.0,-0.50,0.51,0.12,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.70,-43.10,137.63,35.20,100759.0,-55460.0,73210.0,-3.97,19.0,X-SC,18.18,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-2.19,-12.15,41.24,24.08,45992.0,-15427.0,111523.0,-19.29,28.0,X-SC,6.92,91.0,-0.34,0.83,3.42,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,1.51,-16.72,72.09,43.32,97783.0,-27232.0,135640.0,-28.43,51.0,X-SC,8.13,90.0,-0.28,1.00,4.15,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.21,-19.85,38.23,10.78,73632.0,-47715.0,192603.0,-52.19,17.0,X-LC,12.77,1.0,-0.65,1.43,0.21,X40,NTT,FMCG
75,TMPV,600.00,1.18,-29.93,75.52,22.98,119531.0,-67621.0,158277.0,-26.74,36.0,X-LC,3.59,2.0,-0.57,1.17,1.18,XY24,NTT,AUTO
73,TCS,4311.59,0.21,-14.52,38.69,18.55,125095.0,-54911.0,323326.0,-26.21,44.0,X-LC,1.00,3.0,-0.44,2.39,10.00,X40,ATH,IT
81,VBL,671.64,-1.97,-3.15,40.09,35.67,122631.0,-9953.0,305889.0,-14.67,43.0,X-LC,2.90,5.0,-0.08,2.26,10.27,X40N,ATH,FMCG
41,INFY,1972.00,-0.80,12.03,19.83,34.24,69511.0,37646.0,350534.0,-12.78,55.0,X-LC,7.25,6.0,0.54,2.59,21.31,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.31,-32.96,98.24,32.89,50845.0,-25450.0,51756.0,-738.98,55.0,L-MC,10.44,259.0,-0.50,0.38,42.53,XR,NTT,BANKS
17,COALINDIA,484.83,-0.07,2.05,16.83,19.23,25842.0,3090.0,153550.0,19.88,52.0,L-LC,10.52,182.0,0.12,1.14,23.77,XY25,ATH,MINING
5,ASIANPAINT,3460.25,-0.58,-7.03,30.08,20.95,70411.0,-17688.0,234080.0,-7.94,35.0,X-LC,12.60,36.0,-0.25,1.73,26.11,X40,ATH,PAINTS
32,HCLTECH,1908.19,-0.55,8.90,13.72,23.83,36145.0,21522.0,263446.0,13.61,57.0,X-LC,6.33,13.0,0.60,1.95,26.40,X40,ATH,IT
51,NESTLEIND,1377.00,-0.33,13.85,6.63,21.40,20035.0,36762.0,302188.0,5.69,51.0,X-LC,7.04,14.0,1.83,2.24,21.05,XY25,NTT,FMCG


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.81,-31.29,64.31,12.90,96551.0,-68362.0,150134.0,121.15,52.0,H-SC,7.41,173.0,-0.71,1.11,24.84,OX40N,NTT,FINANCE
17,COALINDIA,484.83,-0.07,2.05,16.83,19.23,25842.0,3090.0,153550.0,19.88,52.0,L-LC,10.52,182.0,0.12,1.14,23.77,XY25,ATH,MINING
40,INDUSINDBK,1800.00,0.31,-32.96,98.24,32.89,50845.0,-25450.0,51756.0,-738.98,55.0,L-MC,10.44,259.0,-0.50,0.38,42.53,XR,NTT,BANKS
62,SATIN,274.00,0.19,-17.43,83.89,51.83,191369.0,-48165.0,228119.0,-26.41,55.0,H-SC,3.33,148.0,-0.25,1.69,11.13,XY24,NTT,FINANCE
56,RAJOOENG,143.10,-3.09,-34.15,112.00,39.61,75600.0,-35000.0,67500.0,-55.45,49.0,H-SC,28.73,136.0,-0.46,0.50,9.26,AR,ATH,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.58
1,25,44.82
2,50,76.78


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.50
MC    30.56
LC    26.92
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.27
X40      24.20
X40N     12.84
XY25     10.91
XR        8.76
AR        8.46
OX40N     7.57
SR        0.97
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.16
X-LC    23.49
H-SC    22.78
M-SC    10.99
X-SC     7.96
H-MC     4.57
M-MC     1.45
M-LC     1.24
L-LC     1.14
H-LC     1.05
L-SC     0.77
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.52,-11.29,47.81
IT,12.87,-22.86,84.24
FINANCE,11.08,-19.17,69.18
MISC,7.08,-35.46,90.87
PAINTS,5.77,-18.21,35.77
ELECTRICAL,5.73,-21.02,65.35
INSURANCE,4.78,-5.10,41.06
PHARMA,3.98,-9.48,43.95
BANKS,2.79,-41.61,127.42


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3420130.0,21
AR,1427120.0,10
XR,1309565.0,12
X40,1241452.0,15
X40N,1005632.0,9
OX40N,767652.0,10
XY25,468932.0,7
SR,289713.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3717579.0,24
X-MC,1768943.0,16
M-SC,1571515.0,15
X-LC,1038646.0,13
X-SC,844146.0,8
H-MC,460693.0,3
L-SC,187172.0,2
M-LC,124160.0,1
H-LC,91343.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1302078.0      6
           AR         951831.0      5
M-SC       XY24       892307.0      6
H-SC       XR         846983.0      7
X-MC       X40        602559.0      7
           XY24       470737.0      3
           X40N       448897.0      4
X-LC       X40        440421.0      6
M-SC       OX40N      345625.0      5
X-SC       X40N       329354.0      3
H-SC       OX40N      326974.0      4
X-SC       XY24       316320.0      3
H-SC       SR         289713.0      2
X-MC       XY25       246750.0      2
H-MC       AR         245821.0      2
X-LC       X40N       227381.0      2
           XY24       223816.0      2
H-MC       XY24       214872.0      1
X-SC       X40        198472.0      2
M-SC       XR         195458.0      2
X-LC       XY25       147028.0      3
M-SC       AR         138125.0      2
M-LC       XR         124160.0      1
L-SC       OX40N       95053.0      1
           XR          92119.0      1
H-LC       AR          91343.0      1
L-MC       XR          50845.0      1
M-MC       XY25        49312.0      1
L-LC       XY25        25842.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
